# Working on weather data for a project

[Citrics](https://b.citrics.dev/) is a project that helps people decide before moving to a new city by providing them valuable informations on different cities. One of the core features of the project is being able to get information about top job industries of different cities and compare them. This notebook shows how the data was cleaned, wrangled and new features were created so that they can be used for getting job industry insights.

The data were collected from [Bureau of Labor Statistics (BLS)](https://www.bls.gov/oes/)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# The first dataset is from BLS
# The second dataset contains unique identifiers
# for the 100 cities we worked on

df = pd.read_csv("/content/us_job_industry_data_2019.csv")
df_cities = pd.read_csv("/content/100city_state_data.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head()

,area,area_title,area_type,naics,naics_title,i_group,own_code,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000_orig,loc_quotient,pct_total,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,annual,hourly
0,99,U.S.,1,0,Cross-industry,cross-industry,1235,11-0000,Management Occupations,major,"8,054,120",0.2,NaN,NaN,NaN,58.88,"122,480",0.1,24.03,34.35,50.80,74.16,#,"49,990","71,460","105,660","154,260",#,NaN,NaN
1,99,U.S.,1,0,Cross-industry,cross-industry,1235,13-0000,Business and Financial Operations Occupations,major,"8,183,750",0.2,NaN,NaN,NaN,37.56,"78,130",0.2,18.76,25.06,33.57,45.61,60.60,"39,020","52,130","69,820","94,870","126,040",NaN,NaN
2,99,U.S.,1,0,Cross-industry,cross-industry,1235,15-0000,Computer and Mathematical Occupations,major,"4,552,880",0.4,NaN,NaN,NaN,45.08,"93,760",0.5,21.79,30.22,42.47,57.47,73.08,"45,320","62,850","88,340","119,550","152,010",NaN,NaN
3,99,U.S.,1,0,Cross-industry,cross-industry,1235,17-0000,Architecture and Engineering Occupations,major,"2,592,680",0.5,NaN,NaN,NaN,42.69,"88,800",0.3,21.77,29.28,39.15,52.87,68.56,"45,280","60,910","81,440","109,970","142,610",NaN,NaN
4,99,U.S.,1,0,Cross-industry,cross-industry,1235,19-0000,"Life, Physical, and Social Science Occupations",major,"1,288,920",0.7,NaN,NaN,NaN,37.28,"77,540",0.4,17.62,23.73,32.77,46.24,61.59,"36,640","49,360","68,160","96,180","128,100",NaN,NaN


In [ ]:
# Cleaning up some area titles

df = df.replace(['Phoenix-Mesa-Scottsdale, AZ', 'Los Angeles-Long Beach-Anaheim, CA', 'Killeen-Temple, TX', 'Virginia Beach-Norfolk-Newport News, VA-NC'], ['Phoenix-Mesa-Scottsdale-Chandler-Gilbert, AZ', 'Los Angeles-Long Beach-Anaheim-Glendale-Santa Ana, CA', 'Killeen-Temple-Irving-Plano, TX', 'Virginia Beach-Norfolk-Newport News-Chesapeake, VA-NC'])

In [ ]:
df.shape, df_cities.shape

((395647, 30), (100, 4))

In [ ]:
# Restricting area type to city

df = df[ df["area_type"] == 4]

In [ ]:
df.shape, df_cities.shape

((142410, 30), (100, 4))

In [ ]:
# A function to create city names from area title

def create_cities(areaname):
    #print(areaname)
    areaname = areaname.replace("--","-")
    cities, garbage1 = areaname.split(",")
    areaname = cities.split("-")
    #print(f"{cities} ---- {list_cities}")
    return areaname

In [ ]:
# Applying the function

df['city'] = df.area_title.apply(create_cities)
df.head()

,area,area_title,area_type,naics,naics_title,i_group,own_code,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000_orig,loc_quotient,pct_total,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,annual,hourly,city
37227,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,11-0000,Management Occupations,major,"2,670",5.8,40.137,0.73,NaN,43.51,"90,500",2.2,17.86,27.22,37.51,51.88,75.57,"37,140","56,610","78,030","107,910","157,190",NaN,NaN,[Abilene]
37228,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,13-0000,Business and Financial Operations Occupations,major,"2,280",6.3,34.214,0.61,NaN,32.46,"67,520",2.9,16.39,22.11,29.47,38.38,48.95,"34,090","46,000","61,300","79,840","101,820",NaN,NaN,[Abilene]
37229,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,15-0000,Computer and Mathematical Occupations,major,630,8.5,9.494,0.31,NaN,30.27,"62,970",3.1,16.59,21.44,27.31,36.98,49.45,"34,510","44,600","56,810","76,910","102,860",NaN,NaN,[Abilene]
37230,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,17-0000,Architecture and Engineering Occupations,major,770,16.0,11.600,0.66,NaN,34.83,"72,440",4.6,16.92,23.82,32.11,43.51,56.79,"35,200","49,540","66,790","90,500","118,110",NaN,NaN,[Abilene]
37231,10180,"Abilene, TX",4,0,Cross-industry,cross-industry,1235,19-0000,"Life, Physical, and Social Science Occupations",major,330,11.5,5.033,0.57,NaN,28.06,"58,360",3.1,16.97,20.27,25.94,33.07,41.27,"35,300","42,170","53,960","68,790","85,850",NaN,NaN,[Abilene]


In [ ]:
df_explode = df.explode('city')


In [ ]:
df_explode.shape

(241969, 31)

In [ ]:
df_explode.columns

Index(['area', 'area_title', 'area_type', 'naics', 'naics_title', 'i_group',
       'own_code', 'occ_code', 'occ_title', 'o_group', 'tot_emp', 'emp_prse',
       'jobs_1000_orig', 'loc_quotient', 'pct_total', 'h_mean', 'a_mean',
       'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90',
       'a_pct10', 'a_pct25', 'a_median', 'a_pct75', 'a_pct90', 'annual',
       'hourly', 'city'],
      dtype='object')

In [ ]:
df_explode[['area_title', 'city']].head()

,area_title,city
37227,"Abilene, TX",Abilene
37228,"Abilene, TX",Abilene
37229,"Abilene, TX",Abilene
37230,"Abilene, TX",Abilene
37231,"Abilene, TX",Abilene


In [ ]:
missing = ['Chandler', 'Gilbert', 'Glendale', 'Santa Ana', 'Honolulu', 'Boise', 'Louisville', 'Winston Salem', 'Irving', 'Plano', 'Chesapeake']
sorted(missing)

['Boise',
 'Chandler',
 'Chesapeake',
 'Gilbert',
 'Glendale',
 'Honolulu',
 'Irving',
 'Louisville',
 'Plano',
 'Santa Ana',
 'Winston Salem']

In [ ]:
len(df_explode.city.unique())

564

In [ ]:
len(df_explode.area_title.unique())

396

In [ ]:
# Cleaning up some more city names

df_explode = df_explode.replace(['Boise City', 'Louisville/Jefferson County', 'Urban Honolulu', 'Winston'], ['Boise', 'Louisville', 'Honolulu', 'Winston Salem'])

In [ ]:
# merging with the dataframe with unique identifier

merged = df_cities.merge(df_explode, on="city", how="left")

In [ ]:
merged.head()

,city_id,city,state,city_state,area,area_title,area_type,naics,naics_title,i_group,own_code,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000_orig,loc_quotient,pct_total,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,annual,hourly
0,0,Anchorage,AK,"Anchorage, AK",11260,"Anchorage, AK",4,0,Cross-industry,cross-industry,1235,11-0000,Management Occupations,major,"11,830",3.1,67.816,1.24,NaN,53.69,"111,680",1.1,25.90,35.17,48.07,63.20,87.82,"53,860","73,150","99,970","131,450","182,670",NaN,NaN
1,0,Anchorage,AK,"Anchorage, AK",11260,"Anchorage, AK",4,0,Cross-industry,cross-industry,1235,13-0000,Business and Financial Operations Occupations,major,"7,920",2.9,45.376,0.81,NaN,35.98,"74,840",4.4,15.08,25.79,34.26,44.87,55.86,"31,360","53,650","71,270","93,320","116,180",NaN,NaN
2,0,Anchorage,AK,"Anchorage, AK",11260,"Anchorage, AK",4,0,Cross-industry,cross-industry,1235,15-0000,Computer and Mathematical Occupations,major,"3,280",8.9,18.769,0.61,NaN,39.30,"81,740",2.2,20.86,27.21,37.11,48.15,60.20,"43,380","56,600","77,180","100,140","125,210",NaN,NaN
3,0,Anchorage,AK,"Anchorage, AK",11260,"Anchorage, AK",4,0,Cross-industry,cross-industry,1235,17-0000,Architecture and Engineering Occupations,major,"4,440",6.4,25.447,1.44,NaN,49.13,"102,180",2.7,26.30,35.33,46.06,59.47,75.06,"54,710","73,480","95,790","123,700","156,110",NaN,NaN
4,0,Anchorage,AK,"Anchorage, AK",11260,"Anchorage, AK",4,0,Cross-industry,cross-industry,1235,19-0000,"Life, Physical, and Social Science Occupations",major,"2,930",6.7,16.783,1.91,NaN,38.72,"80,540",3.0,21.06,27.87,36.41,47.04,58.97,"43,810","57,970","75,720","97,830","122,660",NaN,NaN


In [ ]:
merged.shape

(62282, 34)

In [ ]:
merged.isnull().sum()

city_id               0
city                  0
state                 0
city_state            0
area                  0
area_title            0
area_type             0
naics                 0
naics_title           0
i_group               0
own_code              0
occ_code              0
occ_title             0
o_group               0
tot_emp               0
emp_prse              0
jobs_1000_orig        0
loc_quotient       4888
pct_total         62282
h_mean                0
a_mean                0
mean_prse             0
h_pct10               0
h_pct25               0
h_median              0
h_pct75               0
h_pct90               0
a_pct10               0
a_pct25               0
a_median              0
a_pct75               0
a_pct90               0
annual            58201
hourly            62056
dtype: int64

In [ ]:
len(merged.city.unique())

100

In [ ]:
# Downloading the data

merged.to_csv('test1.csv')

To see how the endpoints were created using this data click [here](https://colab.research.google.com/drive/1IJD9aRTVFZiIvYKZztM2xzUEbmkM3Y6E?usp=sharing)